In [ ]:
%pip install scispacy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 16.7 MB/s eta 0:00:00


In [ ]:
%pip install en_core_sci_md
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_web_trf-0.5.4.tar.gz


  ERROR: HTTP error 404 while getting https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_web_trf-0.5.4.tar.gz
ERROR: Could not install requirement https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_web_trf-0.5.4.tar.gz because of HTTP error 404 Client Error: Not Found for url: https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_web_trf-0.5.4.tar.gz for URL https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_web_trf-0.5.4.tar.gz


In [ ]:
import spacy
import scispacy

# Load scispacy model
nlp = spacy.load("en_core_sci_md")


In [ ]:
from flask import Flask, request, jsonify
import spacy
import threading
import subprocess
import time

# Initialize Flask app
app = Flask(__name__) # SciSpacy for medical entities
nlp_medical = spacy.load("en_ner_bc5cdr_md")
nlp_general = spacy.load("en_core_web_trf")  # General NLP for names & ages

def extract_ehr_data(text):
    doc_medical = nlp_medical(text)
    doc_general = nlp_general(text)

    ehr_data = {
        "patient_name": "",
        "age": None,
        "condition": "",
        "procedure": "",
        "medication": "",
    }

    # Extract from general NLP model
    for ent in doc_general.ents:
        if ent.label_ == "PERSON":
            ehr_data["patient_name"] = ent.text
        elif ent.label_ in ["DATE", "CARDINAL"]:  # Sometimes age is labeled as DATE or CARDINAL
            ehr_data["age"] = ent.text

    # Extract from medical NLP model (SciSpacy)
    for ent in doc_medical.ents:
        print(f"Text: {ent.text}, Label: {ent.label_}")  # ✅ Debugging

        if ent.label_ == "PERSON":
            ehr_data["patient_name"] = ent.text
        elif ent.label_ in ["DATE", "CARDINAL", "AGE"]:
            ehr_data["age"] = ent.text
        elif ent.label_ in ["DISEASE", "DISORDER", "CONDITION", "PROBLEM"]:
            ehr_data["condition"] = ent.text  # ✅ Added missing labels
        elif ent.label_ in ["PROCEDURE", "TREATMENT", "THERAPY"]:
            ehr_data["procedure"] = ent.text
        elif ent.label_ in ["MEDICATION", "DRUG", "CHEMICAL"]:
            ehr_data["medication"] = ent.text  # ✅ More medication labels

    return ehr_data


# Define API route
@app.route('/process_text', methods=['POST'])
def process_text():
    input_data = request.json
    text = input_data.get('text')

    if not text:
        return jsonify({"error": "No text provided"}), 400

    ehr_data = extract_ehr_data(text)

    return jsonify(ehr_data)

# Function to start Flask server
def run_flask():
    print("🚀 Flask is running on: http://0.0.0.0:5000")
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

# Function to start Localtunnel
def run_localtunnel():
    time.sleep(3)  # Wait for Flask to start
    print("🌍 Starting Localtunnel...")
    subprocess.run(["npx", "localtunnel", "--port", "5000", "--subdomain", "ehr-api-demo"])

# Start Flask in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Start Localtunnel in a separate thread
tunnel_thread = threading.Thread(target=run_localtunnel)
tunnel_thread.start()


/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map_location=dev

🚀 Flask is running on: http://0.0.0.0:5000
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [ ]:
!npx localtunnel --port 5000 --subdomain ehr-api-demo

⠙⠹⠸⠼⠴your url is: https://stupid-turkey-75.loca.lt


/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
INFO:werkzeug:127.0.0.1 - - [01/Mar/2025 16:01:14] "POST /process_text HTTP/1.1" 200 -


Text: diabetes, Label: DISEASE
Text: metformin, Label: CHEMICAL


INFO:werkzeug:127.0.0.1 - - [01/Mar/2025 16:02:04] "POST /process_text HTTP/1.1" 200 -


Text: pneumonia, Label: DISEASE
Text: azithromycin, Label: CHEMICAL


...

Text: pneumonia, Label: DISEASE
Text: azithromycin, Label: CHEMICAL


INFO:werkzeug:127.0.0.1 - - [01/Mar/2025 17:21:01] "POST /process_text HTTP/1.1" 200 -


Text: pneumonia, Label: DISEASE
Text: azithromycin, Label: CHEMICAL


INFO:werkzeug:127.0.0.1 - - [01/Mar/2025 18:26:08] "POST /process_text HTTP/1.1" 200 -


Text: pneumonia, Label: DISEASE
Text: azithromycin, Label: CHEMICAL


In [ ]:
import spacy

# Load SciSpacy model
nlp = spacy.load("en_core_sci_md")

# Test input
text = "Dr. John Smith, age 45, diagnosed with hypertension and prescribed losartan."

# Process text
doc = nlp(text)

# Print detected entities
for ent in doc.ents:
    print(f"Text: {ent.text}, Label: {ent.label_}")


Text: age, Label: ENTITY
Text: diagnosed, Label: ENTITY
Text: hypertension, Label: ENTITY
Text: prescribed, Label: ENTITY
Text: losartan, Label: ENTITY


In [ ]:
%pip install spacy scispacy
!python -m spacy download en_core_web_trf
!python -m spacy download en_core_sci_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.6/735.6 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import spacy

# Load two models

# Test it
text = "Dr. John Smith, age 45, diagnosed with hypertension and prescribed losartan."
print(extract_ehr_data(text))


Text: Dr. John Smith, Label: ENTITY
Text: age, Label: ENTITY
Text: diagnosed, Label: ENTITY
Text: hypertension, Label: ENTITY
Text: prescribed, Label: ENTITY
Text: losartan, Label: ENTITY
{'patient_name': '', 'age': None, 'condition': '', 'procedure': '', 'medication': ''}


In [ ]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_lg-0.5.4.tar.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.2/531.2 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en_core_sci_lg: filename=en_core_sci_lg-0.5.4-py3-none-any.whl size=531476535 sha256=6b8d669d8bdf49fd11cd5f61ad5cd8405cb8c3dda405160b0f68079f673251f3
  Stored in directory: /root/.cache/pip/wheels/d6/b1/2f/ce1b43e2194a41a5c93baa170fd8cb638e4938a068a7190068
Successfully built en_core_sci_lg


In [ ]:
import spacy

# Load medical NLP model with detailed entities
nlp_medical = spacy.load("en_core_sci_lg")
nlp_general = spacy.load("en_core_web_trf")  # General NLP model

def extract_ehr_data(text):
    doc_medical = nlp_medical(text)
    doc_general = nlp_general(text)

    ehr_data = {
        "patient_name": "",
        "age": None,
        "condition": "",
        "procedure": "",
        "medication": "",
    }

    # Extract from general NLP model (names, ages)
    for ent in doc_general.ents:
        if ent.label_ == "PERSON":
            ehr_data["patient_name"] = ent.text
        elif ent.label_ in ["DATE", "CARDINAL"]:
            ehr_data["age"] = ent.text

    # Extract from medical NLP model (diseases, medications, procedures)
    for ent in doc_medical.ents:
        if ent.label_ in ["DISEASE", "DISORDER", "CONDITION"]:
            ehr_data["condition"] = ent.text
        elif ent.label_ in ["PROCEDURE", "TREATMENT"]:
            ehr_data["procedure"] = ent.text
        elif ent.label_ in ["MEDICATION", "DRUG"]:
            ehr_data["medication"] = ent.text

    return ehr_data

# ✅ Test the function
text = "Dr. John Smith, age 45, diagnosed with hypertension and prescribed losartan."
print(extract_ehr_data(text))


In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_scibert-0.5.4.tar.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.0/417.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 24.3 MB/s eta 0:00:00
  Created wheel for en_core_sci_scibert: filename=en_core_sci_scibert-0.5.4-py3-none-any.whl size=416981900 sha256=547efbcc4cba4417e036ed354dc9f9583195e2b9fe429601e8ef926fa31322a9
  Stored in directory: /root/.cache/pip/wheels/4f/d3/8e/bae45104627bcc6ecfbd308a77df060c56fd50fa1c4851e6c2
Successfully built en_core_sci_scibert


In [ ]:
import spacy

# ✅ Load the best medical model
nlp_medical = spacy.load("en_core_sci_lg")

# ✅ Sample text
text = "John Smith, age 45, diagnosed with hypertension and prescribed losartan."

# ✅ Process text
doc = nlp_medical(text)

# ✅ Print all detected entities
for ent in doc.ents:
    print(f"Text: {ent.text}, Label: {ent.label_}")


/usr/local/lib/python3.11/dist-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


Text: John Smith, Label: ENTITY
Text: age, Label: ENTITY
Text: diagnosed, Label: ENTITY
Text: hypertension, Label: ENTITY
Text: prescribed, Label: ENTITY
Text: losartan, Label: ENTITY


In [ ]:
def extract_ehr_data(text):
    doc = nlp_medical(text)

    ehr_data = {
        "patient_name": "",
        "age": None,
        "condition": "",
        "procedure": "",
        "medication": "",
    }

    for ent in doc.ents:
        print(f"Text: {ent.text}, Label: {ent.label_}")  # ✅ Debugging

        if ent.label_ == "PERSON":
            ehr_data["patient_name"] = ent.text
        elif ent.label_ in ["DATE", "CARDINAL", "AGE"]:
            ehr_data["age"] = ent.text
        elif ent.label_ in ["DISEASE", "DISORDER", "CONDITION", "PROBLEM"]:
            ehr_data["condition"] = ent.text  # ✅ Added missing labels
        elif ent.label_ in ["PROCEDURE", "TREATMENT", "THERAPY"]:
            ehr_data["procedure"] = ent.text
        elif ent.label_ in ["MEDICATION", "DRUG", "CHEMICAL"]:
            ehr_data["medication"] = ent.text  # ✅ More medication labels

    return ehr_data


In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_scibert-0.5.4.tar.gz


  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_scibert-0.5.4.tar.gz (417.0 MB)
  Preparing metadata (setup.py) ... done


In [ ]:
nlp_medical = spacy.load("en_core_sci_scibert")


/usr/local/lib/python3.11/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.load_state_dict(torch.load(fileli

In [ ]:
def extract_ehr_data(text):
    doc = nlp_medical(text)

    ehr_data = {
        "patient_name": "",
        "age": None,
        "condition": "",
        "procedure": "",
        "medication": "",
    }

    for ent in doc.ents:
        print(f"Text: {ent.text}, Label: {ent.label_}")  # ✅ Debugging

        if ent.label_ == "PERSON":
            ehr_data["patient_name"] = ent.text
        elif ent.label_ in ["DATE", "CARDINAL", "AGE"]:
            ehr_data["age"] = ent.text
        elif ent.label_ in ["DISEASE", "DISORDER", "CONDITION", "PROBLEM"]:
            ehr_data["condition"] = ent.text  # ✅ Added missing labels
        elif ent.label_ in ["PROCEDURE", "TREATMENT", "THERAPY"]:
            ehr_data["procedure"] = ent.text
        elif ent.label_ in ["MEDICATION", "DRUG", "CHEMICAL"]:
            ehr_data["medication"] = ent.text  # ✅ More medication labels

    return ehr_data
print(extract_ehr_data(text))

Text: hypertension, Label: DISEASE
Text: losartan, Label: CHEMICAL
{'patient_name': '', 'age': None, 'condition': 'hypertension', 'procedure': '', 'medication': 'losartan'}


In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_ner_bc5cdr_md-0.5.4.tar.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en_ner_bc5cdr_md: filename=en_ner_bc5cdr_md-0.5.4-py3-none-any.whl size=119787678 sha256=385d54529308ae3df9c77cba201f4d81db72ac3772bd59ec97a71d5c9027d3ae
  Stored in directory: /root/.cache/pip/wheels/6e/a6/d6/bd15a41e2ff02a62f0a0a48dddbc07d048307db7199a1538f7
Successfully built en_ner_bc5cdr_md


In [ ]:
nlp_medical = spacy.load("en_ner_bc5cdr_md")
